In [1]:
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
import holidays
import warnings
from merge_transformer import MergeTransformer
from create_db import create_db

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def check_csv_correct(X):
    
    if X.isna().sum().all() != 0:
        return False
    return True

In [3]:
def merge_dfs(database, submission_dir):
    
    X = database['Weather']
    
    merge_transform = MergeTransformer(X_ext=database['Date'], how='left', on=['DateOfDeparture'])
    X = merge_transform.fit_transform(X)
        
    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'AirPort',
                        'latitude_deg': 'latitude_departure',
                        'longitude_deg': 'longitude_departure',
                        'state': 'state_departure',
                        'capacity': 'capacity_departure'},
        how='inner', on=['AirPort'])
    X = merge_transform.fit_transform(X)
    
    merge_transform = MergeTransformer(
        X_ext=database['StateFeatures'],
        cols_to_keep=['DateOfDeparture', 'state', 'UnemploymentRate', 'holidays'],
        cols_to_rename={'state': 'state_departure'}, 
        how='left',
        on=['DateOfDeparture', 'state_departure'])

    X = merge_transform.fit_transform(X)
    
    X.rename({'AirPort': 'Arrival'}, axis=1, inplace=True)
    
    all_check = check_csv_correct(X)
    
    if all_check:
        X.to_csv(submission_dir + 'external_data.csv')
    else:
        raise Exception("Sorry will not create CSV as Database has Nans or some other shit. Try Again!")
        
    return all_check

In [4]:
def create_external_data(submission_dir):
    
    database = create_db()
    merge_dfs(database, submission_dir)

In [5]:
create_external_data('../submissions/test/')